In [36]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

import re
from pymorphy3 import MorphAnalyzer

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline

import os
os.environ["KERAS_BACKEND"] = "torch"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
# torch.set_default_device('cpu')

import keras
print(keras.__version__)

import gensim

3.9.2


In [3]:
wiki = open('wiki_data.txt').read().split('\n')
len(wiki)

20003

# **Задание 1 (3 балла)**
Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:

1. добавьте лемматизацию в предобработку (любым способом)
2. измените размер окна в большую или меньшую сторону
3. измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре). Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

## Предобработка

In [4]:
# глобальные переменные для работы с морфологией
morph_parser = MorphAnalyzer()
cash = {}

# 1. добавление лемматизации
def lemmatize(token):
    global morph_parser
    global cash
    if token in cash:
        lemmatized = cash[token]
    else:
        lemmatized = morph_parser.parse(token)[0].normal_form
        cash[token] = lemmatized

    return lemmatized

def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [lemmatize(token) for token in tokens if token]
    return tokens

In [5]:
vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))

filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)


id2word = {i:word for word, i in word2id.items()}

sentences = []

for text in wiki:
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [6]:
vocab_size = len(id2word)

In [7]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

## CBOW

In [8]:
# 2. уменьшила размерность окна
def gen_batches_cbow(sentences, window=4, batch_size=1000):
    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [9]:
#cbow negative sampling
inputs_target = keras.layers.Input(shape=(1,))
inputs_context = keras.layers.Input(shape=(4,))


embeddings_target = keras.layers.Embedding(input_dim=len(word2id), output_dim=200)(inputs_target, )
embeddings_context = keras.layers.Embedding(input_dim=len(word2id), output_dim=200)(inputs_context, )

target = keras.layers.Flatten()(embeddings_target)

context = keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=1),
                             output_shape=(200,))(embeddings_context)

dot = keras.layers.Dot(1)([target, context])

# полученную близость нужно преобразовать в вероятность
# когда она одна используется не софтмакс и сигмоида
outputs = keras.layers.Activation(activation='sigmoid')(dot)

model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)


optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

2025-04-11 16:03:05.306644: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
model.build([(None, 1), (None, 4)])
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 200)    │  2,492,200 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 4, 200)    │  2,492,200 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 200)       │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 200)       │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ flatten[0][0],    │
│                     │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,984,400 (19.01 MB)

 Trainable params: 4,984,400 (19.01 MB)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
model.fit(gen_batches_cbow(sentences[:19000], window=4),
          validation_data=gen_batches_cbow(sentences[19000:],  window=4),
          batch_size=1000,
          steps_per_epoch=5000,
          validation_steps=30,
          epochs=10)

Epoch 1/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 143s 28ms/step - accuracy: 0.8100 - loss: 0.4217 - val_accuracy: 0.8648 - val_loss: 0.3267
Epoch 2/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 149s 30ms/step - accuracy: 0.8733 - loss: 0.3058 - val_accuracy: 0.8850 - val_loss: 0.2776
Epoch 3/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 149s 30ms/step - accuracy: 0.8975 - loss: 0.2489 - val_accuracy: 0.8870 - val_loss: 0.2742
Epoch 4/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 150s 30ms/step - accuracy: 0.9080 - loss: 0.2250 - val_accuracy: 0.8998 - val_loss: 0.2506
Epoch 5/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 149s 30ms/step - accuracy: 0.9158 - loss: 0.2066 - val_accuracy: 0.8885 - val_loss: 0.2821
Epoch 6/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 148s 30ms/step - accuracy: 0.9253 - loss: 0.1853 - val_accuracy: 0.8894 - val_loss: 0.2861
Epoch 7/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 150s 30ms/step - accuracy: 0.9297 - loss: 0.1760 - val_accuracy: 0.8806 - val_loss: 0.3264
Epoch 8/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 202s 40ms/step - accuracy: 

In [12]:
embeddings = model.layers[2].get_weights()[0]

In [13]:
most_similar('металл', embeddings)

['металл',
 'сплав',
 'водород',
 'железо',
 'полупроводниковый',
 'волокно',
 'оксид',
 'спирт',
 'древесный',
 'медь']

In [14]:
most_similar('рок', embeddings)

['рок',
 'джаз',
 'многоголосный',
 'хореографический',
 'модерн',
 'танец',
 'рей',
 'струнный',
 'фигурист',
 'барокко']

In [15]:
most_similar('небо', embeddings)

['небо',
 'лазоревый',
 'лететь',
 'олень',
 'клюв',
 'дурьодхан',
 'луг',
 'оконечность',
 'намёк',
 'висеть']

## Skip-gram

In [16]:
# 2. уменьшила размерность окна
def gen_batches_sg(sentences, window=4, batch_size=1000):

    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + sent[i+1:i+right_context_length]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [17]:
inputs_target = keras.layers.Input(shape=(1,))
inputs_context = keras.layers.Input(shape=(1,))

# 3. уменьшила размерность векторов
embeddings_target = keras.layers.Embedding(input_dim=len(word2id), output_dim=200)(inputs_target, )
embeddings_context = keras.layers.Embedding(input_dim=len(word2id), output_dim=200)(inputs_context, )

target = keras.layers.Flatten()(embeddings_target)
context = keras.layers.Flatten()(embeddings_context)

dot = keras.layers.Dot(1)([target, context])
outputs = keras.layers.Activation(activation='sigmoid')(dot)

model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
model.build((None, 1))
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 200)    │  2,492,200 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 200)    │  2,492,200 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 200)       │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 200)       │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 1)         │          0 │ dot_1[0][0]       │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,984,400 (19.01 MB)

 Trainable params: 4,984,400 (19.01 MB)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
model.fit(gen_batches_sg(sentences[:19000], window=4),
          validation_data=gen_batches_sg(sentences[19000:],  window=4),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
          epochs=10)

Epoch 1/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 283s 28ms/step - accuracy: 0.7882 - loss: 0.4590 - val_accuracy: 0.8210 - val_loss: 0.4107
Epoch 2/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 284s 28ms/step - accuracy: 0.8429 - loss: 0.3680 - val_accuracy: 0.8428 - val_loss: 0.3634
Epoch 3/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 278s 28ms/step - accuracy: 0.8468 - loss: 0.3589 - val_accuracy: 0.8506 - val_loss: 0.3608
Epoch 4/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 271s 27ms/step - accuracy: 0.8550 - loss: 0.3389 - val_accuracy: 0.8593 - val_loss: 0.3270
Epoch 5/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 275s 27ms/step - accuracy: 0.8558 - loss: 0.3374 - val_accuracy: 0.8460 - val_loss: 0.3627
Epoch 6/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 278s 28ms/step - accuracy: 0.8605 - loss: 0.3260 - val_accuracy: 0.8527 - val_loss: 0.3540
Epoch 7/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 273s 27ms/step - accuracy: 0.8600 - loss: 0.3255 - val_accuracy: 0.8663 - val_loss: 0.3176
Epoch 8/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 274s 27ms/s

In [20]:
embeddings = model.layers[2].get_weights()[0]

In [21]:
most_similar('металл', embeddings)

['металл',
 'оксид',
 'азотный',
 'ванадий',
 'медь',
 'сплав',
 'свинец',
 'кальций',
 'жидкий',
 'пигмент']

In [22]:
most_similar('рок', embeddings)

['рок',
 'джаз',
 'дуэт',
 'инструментальный',
 'жанр',
 'дэйв',
 'маккартень',
 'клавишный',
 'танцевальный',
 'вокальный']

In [23]:
most_similar('небо', embeddings)

['небо',
 'небесный',
 'венера',
 'солнце',
 'пыль',
 'кронос',
 'луч',
 'дева',
 'божество',
 'бог']

# **Задание 2 (2 балла)**
Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [24]:
texts = [preprocess(text) for text in wiki]

## Word2Vec

In [49]:
%%time
word2vec = gensim.models.Word2Vec(texts, 
                                  negative=3,
                                  ns_exponent=0.89,
                                  sample=1e-5,
                                  hs=0,
                                  sg=1,
                                  epochs=10,
                                  max_vocab_size=12000,
                                  window=4,
                                  vector_size=400, 
                                  min_count=40)

CPU times: user 37.9 s, sys: 158 ms, total: 38.1 s
Wall time: 13.6 s


In [50]:
word2vec.wv.most_similar('отец')

[('сестра', 0.9184367656707764),
 ('мать', 0.9029901623725891),
 ('брат', 0.8839795589447021),
 ('сын', 0.8640981316566467),
 ('жениться', 0.853570818901062),
 ('дочь', 0.8474732637405396),
 ('муж', 0.8423340320587158),
 ('родить', 0.8414931893348694),
 ('жена', 0.8367605209350586),
 ('супруг', 0.8155689835548401)]

In [65]:
word2vec.wv.most_similar('дерево')

[('питаться', 0.8896279335021973),
 ('рыба', 0.8715323805809021),
 ('растение', 0.8712673187255859),
 ('семя', 0.8697709441184998),
 ('птица', 0.8651474714279175),
 ('покрыть', 0.8550669550895691),
 ('обитать', 0.855018675327301),
 ('яйцо', 0.8484337329864502),
 ('змея', 0.847160279750824),
 ('цветок', 0.8437319993972778)]

## FastText

In [57]:
%%time
fasttext = gensim.models.FastText(texts, 
                             vector_size=200, 
                             min_count=40, 
                             max_vocab_size=9000,
                             window=4,
                             epochs=10,
                             min_n = 4,
                             max_n = 9) 

CPU times: user 4min 57s, sys: 9.24 s, total: 5min 6s
Wall time: 1min 53s


In [58]:
fasttext.wv.most_similar('отец')

[('мать', 0.7916831970214844),
 ('сын', 0.7409446239471436),
 ('родитель', 0.7313957810401917),
 ('сестра', 0.7273893356323242),
 ('брат', 0.7196367979049683),
 ('жена', 0.7052701711654663),
 ('супруг', 0.6896678805351257),
 ('дочь', 0.6890468001365662),
 ('муж', 0.6876423954963684),
 ('родственник', 0.6423093676567078)]

In [67]:
fasttext.wv.most_similar('дерево')

[('лес', 0.6922392845153809),
 ('камень', 0.6807040572166443),
 ('птица', 0.6297154426574707),
 ('растение', 0.6289269328117371),
 ('почва', 0.6141160726547241),
 ('лист', 0.6127113699913025),
 ('покрыть', 0.6040641069412231),
 ('рыба', 0.6028006672859192),
 ('яйцо', 0.5870954394340515),
 ('плод', 0.5765817761421204)]

# **Задание 3 (3 балла)**
Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.

В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)

Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор. В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)
F1 мера должна быть выше 20%.

## Подготовка данных

In [68]:
data = pd.read_csv('labeled.csv')
data['norm_text'] = data.comment.apply(preprocess)
data.head()

,comment,toxic,norm_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд-то, за, что, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\n,1.0,"[собака, собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, тоже, не, оск..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убедить, 6-страничный, пдф, в, тот, ч..."


In [105]:
%%time

# т. к. в корпусе достаточно много специфической лексики,
# которая не была представлена в материале по википедии, то есть смысл
# обучить word2vec заново
texts_tox = [com for com in data['norm_text']]
word2vec_tox = gensim.models.Word2Vec(texts_tox, 
                                  epochs=10,
                                  max_vocab_size=14000,
                                  window=4,
                                  vector_size=300, 
                                  min_count=20)

CPU times: user 3.67 s, sys: 21.6 ms, total: 3.69 s
Wall time: 1.36 s


In [117]:
word2vec_tox.wv.most_similar('президент')

[('израиль', 0.9537786841392517),
 ('военный', 0.9438151717185974),
 ('согласно', 0.9414976835250854),
 ('ранее', 0.9314428567886353),
 ('глава', 0.9289054870605469),
 ('экономический', 0.9255659580230713),
 ('вашингтон', 0.9248909950256348),
 ('21', 0.921998143196106),
 ('лидер', 0.9203082323074341),
 ('служба', 0.920089602470398)]

In [127]:
# функция для усреднения векторов
def get_text_embeddings(text):
    global word2vec_tox
    embeddings = list()
    for word in text:
        try:
            embeddings.append(word2vec_tox.wv[word])
        except:
            pass
    
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_tox.vector_size)

In [133]:
data['embeddings'] = data.comment.apply(get_text_embeddings)

In [137]:
# проверим, что все прошло по плану

for array in data['embeddings']:
    if array.shape != (300,):
        print(array.shape)

In [187]:
X = np.vstack(data.embeddings)
y = data.toxic.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Обучение классификатора

In [188]:
# обучаем классификатор
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [189]:
model_tox_classifier = LogisticRegression()
model_tox_classifier.fit(X_train, y_train)

LogisticRegression()

In [190]:
preds = model_tox_classifier.predict(X_test)
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

         0.0       0.95      0.72      0.82       652
         1.0       0.19      0.62      0.29        69

    accuracy                           0.71       721
   macro avg       0.57      0.67      0.56       721
weighted avg       0.88      0.71      0.77       721



# **Задание 4 (2 доп балла)**

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.